PYOMO


In [1]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [2]:
working_in_colab = True # Set this variable equal to True if you are working in Colab, False otherwise

if working_in_colab:
    # WARNING: On Colab only
    # This needs to be done once at the start of each session.
    !pip install -q pyomo              # Quiet installation of pyomo using pip.
    !apt-get install -y -qq coinor-cbc # Installation of COIN-OR CBC.

ERROR: Invalid requirement: '#'
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
if working_in_colab:
    cbc_path = "/usr/bin/cbc"

In [5]:
from pyomo.environ  import *

In [6]:
import pandas as pd

cost = pd.read_csv("cost_vector.csv", index_col= 0 )
ebitda = pd.read_csv("ebitda_matrix.csv",index_col= 0)


matrix = pd.read_csv("/x_matrix.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'cost_vector.csv'

In [122]:
model = ConcreteModel()

num_y = cost.size

num_rows = ebitda.shape[0]
num_cols = ebitda.shape[1]

# Create x
model.rows = RangeSet(1,num_rows)
model.cols = RangeSet(1,num_cols)

# Define decision variables
# model.x = Var(model.rows, model.cols, within = Binary)
model.y = Var(range(1, num_y + 1), within = Binary)

# Define the objective function programmatically
def profit_function(model):
    # Calculate total cost
    cost_total = sum(model.y[j] * cost.iloc[j - 1, 0] for j in range(1, num_y + 1))

    # Calculate total EBITDA
    ebitda_total = sum(model.y[j] * ebitda.iloc[i - 1, j - 1] * matrix.iloc[i - 1, j - 1] for i in model.rows for j in range(1, num_y + 1))

    # Calculate profit
    profit_total = ebitda_total - cost_total
    return profit_total

model.obj = Objective(rule = profit_function, sense = maximize)

# Define constraints
# model.constraints = ConstraintList()
# for i in range(1,num_rows):
#     model.constraints.add(sum(model.x[i,j] for j in range(1,num_cols)) == 1)



In [142]:
from pyomo.environ import SolverFactory
solver = SolverFactory('cbc')
results = solver.solve(model, tee = True)

franco = []
for i in range(1, num_cols+1):
    franco.append(model.y[i]())

# # Check if optimization was successful
# if results.solver.termination_condition == TerminationCondition.optimal:
#     print("Optimal Solution Found:")
#     for i in range(1, num_y + 1):
#         print(f"y{i} = {value(model.y[i])}")
#     for i in matrix:
#         for j in matrix:
#             print(f"x[{i}, {j}] = {value(matrix[i, j])}")


Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp4q0o1_g5.pyomo.lp -stat=1 -solve -solu /tmp/tmp4q0o1_g5.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 0 (-1) rows, 0 (-76) columns and 0 (-1) elements
Statistics for presolved model
Original problem has 75 integers (75 of which binary)


Problem has 0 rows, 0 columns (0 with objective) and 0 elements
There are 23727 singletons with no objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G ot

In [141]:
terreni = pd.read_csv("/content/df_prop.csv")

,Unnamed: 0,Title,Price,Surface m^2,Latitude,Longitude
0,0,"cascina cassinazza, pantigliate",557550,35.998,45.434641,9.370862
1,1,"via xx settembre, serle",330000,25.500,45.556050,10.380836
2,2,"via franciacorta 74, , rovato",2630000,50.000,45.574709,10.002811
3,3,"via franciacorta, , rovato",2100000,30.000,45.574709,10.002811
4,4,"via casaglia, , travagliato",3440000,22.900,45.516978,10.088015
...,...,...,...,...,...,...
70,70,"via antonio gramsci 1, , busto garolfo",1350000,22.500,45.544351,8.877002
71,71,"via monte grappa 11, , castel d'ario",950000,26.200,45.194856,10.979426
72,72,"via ferdinando magellano 8, casone, marcallo ...",1500000,35.650,45.495917,8.882195
73,73,"strada provinciale goitese, medole",1950000,20.000,45.336935,10.549414


In [147]:
terreni[pd.Series(franco) == 1].shape

(38, 6)